In [ ]:
import os
import sys
#define path to image directory
image_dir = '/zimages'
#define path to output json file directory
json_dir = '/json'

#check if image directory exists
if not os.path.exists(image_dir):
    print(f"Image directory '{image_dir}' does not exist.")
    sys.exit(1)
#check if json directory exists
if not os.path.exists(json_dir):
    print(f"JSON directory '{json_dir}' does not exist.")
    sys.exit(1)
#check if image directory is empty
if not os.listdir(image_dir):
    print(f"Image directory '{image_dir}' is empty.")
    sys.exit(1)

# check if json directory is empty
if not os.listdir(json_dir):
    print(f"JSON directory '{json_dir}' is empty.")
    sys.exit(1)

# load images sequentialy in an interface that allows marking keypoints (according to coco dataset).
# and then add the keypoints to the json file in the json directory.
# The json file should be in the format of the coco dataset.
# create the basic structure of the json file
import json
from datetime import datetime
from pathlib import Path

def create_coco_json(image_dir, json_dir):
    json_data = {
        "info": {
            "description": "Keypoint annotations",
            "url": "",
            "version": "1.0",
            "year": datetime.now().year,
            "contributor": "",
            "date_created": datetime.now().isoformat()
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }
    return json_data

def add_image(json_data, image_path, sort='desc', *ilocation):
    # Check if the image is already in the JSON data
    # check if ilocation exists, if so slice the image_path
    # retrieve list with image_names in image_dir
    image_names = [Path(image).name for image in os.listdir(image_dir)]

In [33]:
import os
image_dir = 'zimages'
image_path = os.path.join(image_dir, 'Squat_1.jpg')

def annotate_coco_keypoints(image_path, save_to=None):
    """
    Launch an external window to annotate 17 COCO keypoints.
    
    Args:
        image_path (str): Path to the image file.
        save_to (str or None): Path to save COCO JSON. If None, doesn't save.
    
    Returns:
        dict: COCO-format annotation dictionary.
    """
    import matplotlib
    matplotlib.use('TkAgg')  # or 'QtAgg' depending on your system

    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import json
    import os

    COCO_KEYPOINT_NAMES = [
        "nose", "left_eye", "right_eye", "left_ear", "right_ear",
        "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
        "left_wrist", "right_wrist", "left_hip", "right_hip",
        "left_knee", "right_knee", "left_ankle", "right_ankle"
    ]

    clicked_keypoints = []
    current_index = [0]

    img = mpimg.imread(image_path)
    image_height, image_width = img.shape[:2]

    def onclick(event):
        if current_index[0] >= len(COCO_KEYPOINT_NAMES):
            return

        if event.xdata is not None and event.ydata is not None:
            x, y = float(event.xdata), float(event.ydata)
            clicked_keypoints.extend([x, y, 1])  # visibility = 2 (labeled & visible)

            plt.plot(x, y, 'ro')
            plt.text(x, y, COCO_KEYPOINT_NAMES[current_index[0]], color='white', fontsize=8)
            current_index[0] += 1

            if current_index[0] < len(COCO_KEYPOINT_NAMES):
                plt.title(f"Click: {COCO_KEYPOINT_NAMES[current_index[0]]}")
            else:
                plt.title("Done! Closing the window in 1 seconds... or press any key to exit.")
                plt.pause(1)
                plt.close()
                # exit the function
                return
            plt.draw()

    # Create figure in an external window
    fig, ax = plt.subplots()
    ax.imshow(img)
    plt.title(f"Click: {COCO_KEYPOINT_NAMES[current_index[0]]}")
    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show(block=True)  # Block until window is closed

    # Build COCO-style dict
    annotation = {
        "info": {
            "description": "Manual COCO keypoint annotation",
            "version": "1.0"
        },
        "images": [
            {
                "id": 1,
                "file_name": os.path.basename(image_path),
                "width": image_width,
                "height": image_height
            }
        ],
        "annotations": [
            {
                "id": 1,
                "image_id": 1,
                "category_id": 1,
                "keypoints": clicked_keypoints,
                "num_keypoints": len(COCO_KEYPOINT_NAMES),
                "bbox": [0, 0, 0, 0],
                "iscrowd": 0,
                "area": 0
            }
        ],
        "categories": [
            {
                "id": 1,
                "name": "person",
                "supercategory": "person",
                "keypoints": COCO_KEYPOINT_NAMES,
                "skeleton": []
            }
        ]
    }

    if save_to:
        with open(save_to, 'w') as f:
            json.dump(annotation, f, indent=2)
        print(f"✅ Annotation saved to {save_to}")

    return annotation


annotate_coco_keypoints(image_path, save_to=None)

{'info': {'description': 'Manual COCO keypoint annotation', 'version': '1.0'},
 'images': [{'id': 1,
   'file_name': 'Squat_1.jpg',
   'width': 1080,
   'height': 1920}],
 'annotations': [{'id': 1,
   'image_id': 1,
   'category_id': 1,
   'keypoints': [581.0584415584412,
    760.0194805194801,
    1,
    664.1753246753244,
    973.0064935064931,
    1,
    658.9805194805192,
    978.2012987012984,
    1,
    658.9805194805192,
    978.2012987012984,
    1],
   'num_keypoints': 17,
   'bbox': [0, 0, 0, 0],
   'iscrowd': 0,
   'area': 0}],
 'categories': [{'id': 1,
   'name': 'person',
   'supercategory': 'person',
   'keypoints': ['nose',
    'left_eye',
    'right_eye',
    'left_ear',
    'right_ear',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle'],
   'skeleton': []}]}

In [19]:
def random_sample(video_path, output_path, n_samples=5, spread='rand'):
    """
    Randomly samples frames from a video file.
    :param video_path: Path to the video file.
    :param n_samples: Number of frames to sample.
    :param spread: Method of sampling ('rand' for random, 'even' for evenly spaced).
    :return: List of sampled frames.
    """
    import cv2
    import random
    import os
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if n_samples > total_frames:
        raise ValueError("Number of samples exceeds total frames in the video.")
    if spread == 'rand':
        frame_indices = sorted(random.sample(range(total_frames), n_samples))
    elif spread == 'even':
        frame_indices = [i * (total_frames // n_samples) for i in range(n_samples)]
    else:
        raise ValueError("Invalid spread method. Use 'rand' or 'even'.")
    sampled_frames = []
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            sampled_frames.append(frame)

    cap.release()
    # save the sampled frames as .jpg files in the output path
    video_name = os.path.basename(video_path).split('.')[0]
    for i, frame in enumerate(sampled_frames):
        cv2.imwrite(os.path.join(output_path, f"{video_name}_{i+1}.jpg"), frame)
        
    print(f"Sampled {n_samples} frames from {video_path} and saved to {output_path}.")

def clear_images(video_name, image_dir):
    """
    Clears images from the image directory.
    :param video_name: Name of the video file.
    :param image_dir: Path to the image directory.
    """
    import os
    count = 0
    for file in os.listdir(image_dir):
        if file.startswith(video_name):
            os.remove(os.path.join(image_dir, file))
            count += 1
    if count == 0:
        print(f"No images found for video {video_name} in {image_dir}.")
    else:
        print(f"Removed {count} images from {image_dir} for video {video_name}.")

In [25]:
import os
video_dir = 'demo/video'
output_path = 'zimages'
for video in os.listdir(video_dir):
    if not video.endswith(('.mp4', '.avi', '.mov')):
        print(f"Skipping non-video file: {video}")
        continue
    video_path = os.path.join(video_dir, video)

    # check if the output path exists, if not create it
    if not os.path.exists('zimages'):
        os.makedirs('zimages')
    # call the random_sample function
    random_sample(video_path, 'zimages', n_samples=3, spread='even')


Sampled 3 frames from demo/video\sample_video.mp4 and saved to zimages.
Sampled 3 frames from demo/video\Squat.mp4 and saved to zimages.
